### CS-500 Homework 3 

- Contains 2 problems, 50 pts each

## Homework Instructions

1. Begin this homework by modifying the variable DREXEL_USERNAME in the next cell
    * Set the variable to your Drexel username (e.g., "db59")
2. Run all cells in this notebook before proceeding with solving problems
    * Select "Cell" -> "Run All" from the top menu bar
3. Follow instructions supplied with each problem
4. Make sure you **save your work in this notebook regularly** by pressing `CTRL+S` (`CMD+S` on macOS)
5. Submit this notebook with your saved answers

In [1]:
### CHANGE THIS TO YOUR DREXEL USERNAME!
DREXEL_USERNAME="mht47"

In [2]:
## This cell installs python (pypi) packages, then imports modules referenced in this notebook
## Make sure this cell is executed first, before you run any cell in this notebook
import ipydeps
ipydeps.pip(["wand", "json", "lxml", "redis", "sqlite3", "pandas", "numpy", "pillow", "bokeh", "sqlalchemy"])
from IPython.display import IFrame    
from wand.image import Image as WImage
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import redis

In [3]:
### we are going to use a jupyter's magick commands %sql or %%sql to run queries
##  install pip package for the sql magick:
ipydeps.pip([ "ipython-sql", "jupyter-sql", "jupyter_contrib_nbextensions", "rejson"])  

## load sql magick (ipython extension)
%reload_ext sql

## %sql and %%sql magics will return pd.DataFrame objects
%config SqlMagic.autopandas=True 

## display the first 7 rows of pd.DataFrame objects
pd.set_option("max_rows", 11)

from rejson import Client, Path


# Storing routes in Redis 

In this exercise, we will store routes between the airports in Redis using rejson library (document store).
The data schema will support querying for the cheapest route with up-to two stops (connections). 
We will store routes leaving each airport in a separate redis key. And all of destinations that are
reachable with up-to two connections (stops) are organized into a dictionary. We only store the cheapest 
price option!

For example, the following dictionary would be used to store routes leaving Philadelphia as a dictionary under Redis key `PHL`. A direct route from `PHL` to `LAX` priced at `$234.56`. A route from `PHL` to `SFO` for `$123.45` with a stop at `JFK`, and a direct route from `PHL` to `JFK`. 
A route with two stops from `PHL` to `ANC` priced at `$323.45` 

**Please see a working example in the next cell.**

```python

phl_routes = {
    ...
    'ANC': (323.45, 'SFO', 'JFK'),
    'SFO': (123.45, 'JFK'),
    'JFK': (23.45,  ),
    'LAX': (234.56, )
    ...
}

```





In [4]:
from rejson import Client, Path
rj = Client(host='localhost', port=6379, decode_responses=True)

phl_routes = {
    'SFO': (123.45, 'AA', 'JFK'),
    'JFK': (23.45,  'AA'),
    'LAX': (234.56, 'AA',)
}

rj.jsonset('PHL', Path.rootPath(), phl_routes)
rj.jsonget('PHL', Path.rootPath())


{'SFO': [123.45, 'AA', 'JFK'], 'JFK': [23.45, 'AA'], 'LAX': [234.56, 'AA']}

In [5]:
### For this homework, you should only use the data from the airfare table of the routes.v2.db database.
### Your code in this homework does not need to handle data from other tables in routes.v2.db 
### (e.g., routes or airports tables). 
%sql sqlite:///routes.v2.db SELECT * FROM airfare where source='LAX';


Done.


,Airline,Source,Destination,Distance,Price
0,4B,LAX,LAS,233.905549,26.522832
1,AA,LAX,ABQ,813.403797,59.065235
2,AA,LAX,ANC,2499.355589,161.554803
3,AA,LAX,AUS,1430.317784,242.946584
4,AA,LAX,BDL,3142.729230,369.225430
...,...,...,...,...,...
292,WN,LAX,STL,1931.840407,167.877118
293,WN,LAX,TUS,517.075136,107.076260
294,ZK,LAX,MCE,184.016741,44.181214
295,ZK,LAX,PRC,413.088368,84.292209


In [6]:
### PROBLEM 1. [50 pts]: Insert routes into Redis database.
### Implement insert_routes in Python.

def insert_routes():
    """
    Transfer all routes from sqlite:///routes.v2.db 
    into Redis using data schema described above (i.e., cheapest routes with up-to two stops must be considered). 

    Returns
    -------
    status : bool
             returns True if data transferred successfully from SQL to Redis
             returns False otherwise
             
    """
    
    return None


In [7]:
### problem 1

import pandas as pd

from sqlalchemy import create_engine
engine = create_engine('sqlite:///routes.v2.db', echo=False)

from rejson import Client, Path
rj = Client(host='localhost', port=6379, decode_responses=True)

def insert_routes():
    """
    Transfer all routes from sqlite:///routes.v2.db 
    into Redis using data schema described above (i.e., cheapest routes with up-to two stops must be considered). 

    Returns
    -------
    status : bool
             returns True if data transferred successfully from SQL to Redis
             returns False otherwise
             
    """
    
    pd.read_sql(
            """
            drop table if exists two_stop;
            """, engine)
    
    pd.read_sql(
            """
            create table two_stop as
            select
                a1.source init
                ,a2.source layover_1
                ,a2.destination layover_2
                ,a3.destination destination
                ,sum(a1.distance + a2.distance + a3.distance) as distance
                ,sum(a1.price + a2.price + a3.price) as price
            from airfare a1
            join airfare a2 on a1.destination = a2.source
            join airfare a3 on a2.destination = a3.source
            group by 
                a1.source
                ,a2.source
                ,a2.destination
                ,a3.destination;
            """, engine)
    
    pd.read_sql(
            """
            drop table if exists one_stop;
            """, engine)
    
    pd.read_sql(
            """
            create table one_stop as 
                select
                a1.source init
                ,a2.source layover_1
                ,null as layover_2
                ,a2.destination destination
                ,sum(a1.distance + a2.distance) as distance
                ,sum(a1.price + a2.price) as price
            from airfare a1
            join airfare a2 on a1.destination = a2.source
            group by 
                a1.source
                ,a2.source
                ,a2.destination;
            """, engine)
    
    pd.read_sql(
            """
            drop table if exists no_stop;
            """, engine)
    
    pd.read_sql(
            """
            create table no_stop as
            select
                a1.source init
                ,null as layover_1
                ,null as layover_2
                ,a1.destination destination
                ,a1.distance
                ,a1.price
            from airfare a1;
            """, engine)
    
    pd.read_sql(
            """
            drop table if exists cheapest_route;
            """, engine)
    
    pd.read_sql(
            """
            create table cheapest_route as
            select
                init
                ,layover_1
                ,layover_2
                ,destination
                ,distance
                ,min(price) as price
            from
            (
                select
                    init
                    ,layover_1
                    ,layover_2
                    ,destination
                    ,distance
                    ,min(price) as price
                from two_stop
                group by init, destination

                union

                select
                    init
                    ,layover_1
                    ,layover_2
                    ,destination
                    ,distance
                    ,min(price) as price
                from one_stop
                group by init, destination

                union

                select
                    init
                    ,layover_1
                    ,layover_2
                    ,destination
                    ,distance
                    ,min(price) as price
                from no_stop
                group by init, destination
            )
            group by init, destination;
            """, engine)
    
    airports = pd.read_sql("""select distinct source from airfare; """, engine)

    try:
        for n in range(len(airports)):
            item = airports.iat[n,0]

            stage = pd.read_sql(
                    """
                    select destination, price, layover_1, layover_2 from cheapest_route where init = :airport;
                    """, engine, params={'airport' : item})

            routes_dict = stage.set_index('destination').T.to_dict('list')

            rj.jsonset(item, Path.rootPath(), routes_dict)
            rj.jsonget(item, Path.rootPath())
        
        return True
    
    except:
        return False

In [8]:
### PROBLEM 2. [50 pts]: Find cheapest route.
### Implement find_cheapest_route in Python. 

from rejson import Client, Path
rj = Client(host='localhost', port=6379, decode_responses=True)

def find_cheapest_route(rj_client, source, destination):
    """
    Find the cheapest route with up-to two stops between source and destination airports.
    Use existing connection to Redis (rj_client) via rejson library. 
    
    The function returns the total price of the cheapest route 
    between two airports. Routes with up-to two connections (stops) must be considered.
    The function returns None if no route is found.
    
    Parameters
    ----------
    
    rj_client : rejson.Client
              connection to Redis server via rejson library

    source : string
          source city 
          
    destination : string
          destination city 

    Returns
    -------
    price : float
             returns the total price of the route
             returns None if no route can be found between two airports
             
    """
    rj = rj_client
    departures_dict = rj.jsonget(source,Path.rootPath())
    
    try:
        print(departures_dict[destination])
    except:
        return None

In [9]:
%%sql sqlite:///routes.v2.db
/*
drop table if exists two_stop;
create table two_stop as
select
a1.source init
,a2.source layover_1
,a2.destination layover_2
,a3.destination destination
,sum(a1.distance + a2.distance + a3.distance) as distance
,sum(a1.price + a2.price + a3.price) as price
from airfare a1
join airfare a2 on a1.destination = a2.source
join airfare a3 on a2.destination = a3.source
group by 
    a1.source
    ,a2.source
    ,a2.destination
    ,a3.destination;
    
drop table if exists one_stop;
create table one_stop as 
    select
    a1.source init
    ,a2.source layover_1
    ,null as layover_2
    ,a2.destination destination
    ,sum(a1.distance + a2.distance) as distance
    ,sum(a1.price + a2.price) as price
from airfare a1
join airfare a2 on a1.destination = a2.source
group by 
    a1.source
    ,a2.source
    ,a2.destination;

drop table if exists no_stop;
create table no_stop as
select
a1.source init
,null as layover_1
,null as layover_2
,a1.destination destination
,a1.distance
,a1.price
from airfare a1;
*/


0 rows affected.


""


In [10]:
%%sql sqlite:///routes.v2.db

/*drop table if exists cheapest_route;
create table cheapest_route as

select
init
,layover_1
,layover_2
,destination
,distance
,min(price) as price
from (

select
init
,layover_1
,layover_2
,destination
,distance
,min(price) as price
from two_stop
group by init, destination

union

select
init
,layover_1
,layover_2
,destination
,distance
,min(price) as price
from one_stop
group by init, destination

union

select
init
,layover_1
,layover_2
,destination
,distance
,min(price) as price
from no_stop
group by init, destination
)
group by init, destination;*/

0 rows affected.


""


In [11]:
#import pandas as pd

#from sqlalchemy import create_engine
#engine = create_engine('sqlite:///routes.v2.db', echo=False)

#from rejson import Client, Path
#rj = Client(host='localhost', port=6379, decode_responses=True)

#airports = pd.read_sql("""select distinct source from airfare; """, engine)

#for n in range(len(airports)):
#    item = airports.iat[n,0]
#
#    stage = pd.read_sql(
#            """
#            select destination, price, layover_1, layover_2 from cheapest_route where init = :airport;
#            """, engine, params={'airport' : item})
#
#    routes_dict = stage.set_index('destination').T.to_dict('list')
#
#    rj.jsonset(item, Path.rootPath(), routes_dict)
#    rj.jsonget(item, Path.rootPath())



In [12]:

#from rejson import Client, Path
#rj = Client(host='localhost', port=6379, decode_responses=True)

#rj.jsonget(airports.iat[len(airports)-1,0],Path.rootPath())



In [13]:

#import pandas as pd

#from sqlalchemy import create_engine
#engine = create_engine('sqlite:///routes.v2.db', echo=False)

#pd.read_sql(
#            """
#            select * from cheapest_route where layover_2 is null;
#            """, engine)